In [ ]:
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import pickle

with open('../paper_calculations/fork_aligned_reconstruction_wo_tw_errors.pickle', 'rb') as handle:
      aligned_reconstruction_wo_tw_errors = pickle.load(handle)
        
keys = [a[1] for a in aligned_reconstruction_wo_tw_errors]
values = [a[2:] for a in aligned_reconstruction_wo_tw_errors]

aligned_notw_errors_dict = dict(zip(keys,values))

In [ ]:
def clean(rawdata):
    return rawdata.replace("'","\"").replace("False","0").replace("True","1").replace("None","\"None\"")

In [ ]:
def safe_check_required_key_value(dictionary, key, value):
    return key in dictionary.keys() and dictionary[key] == value

In [ ]:
def match_dict_values(curdict, goaldict, ignore_keys_list):
    for key in goaldict.keys():
        if key not in ignore_keys_list:
            if not safe_check_required_key_value(curdict, key, goaldict[key]):
                return False
    return True

In [ ]:
def get_all_data():
    rawdata1 = pd.read_csv("../fork_project.csv")
    rawdata1 = rawdata1.reset_index()  # make sure indexes pair with number of rows

    rawdata2 = pd.read_csv("../forkprojectrateinvariantvae.csv")
    rawdata2 = rawdata2.reset_index()  # make sure indexes pair with number of rows


    rawdata = pd.concat([rawdata1,rawdata2])
    rawdata.reset_index()
    return rawdata

In [ ]:
all_run_dicts = []
rawdata = get_all_data()
for index, row in rawdata.iterrows():
    configdict = json.loads(clean(row.config))
    if not any([match_dict_values(configdict, prd,["logname","model_save_dir","latent_dim"]) for prd in all_run_dicts]):
        all_run_dicts.append(configdict)

In [ ]:
                ("decoder_name","functional_decoder_complicated"),
                ("dec_complicated_function_hidden_dims",[200]),
                ("training_data_added_timing_noise",0.1)

In [ ]:
def create_dataset_combine(sweep_dictionaries,ignore_keys_list = ["logname","model_save_dir","latent_dim"]):
    all_data = []
    for sweep_dictionary in sweep_dictionaries:
        data = create_dataset(sweep_dictionary,ignore_keys_list)
        all_data.append(data)
    all_data = np.concatenate(all_data, axis=0)
    return all_data

In [ ]:
def create_dataset(sweep_dictionary,ignore_keys_list = ["logname","model_save_dir","latent_dim"], minstamp = None):
    data = []
    rawdata = get_all_data()

    for index, row in rawdata.iterrows():
        configdict = json.loads(clean(row.config))
        if match_dict_values(configdict,sweep_dictionary,ignore_keys_list):
            rowdata = json.loads(clean(row.summary))
            if "train_alignedRMSE" in rowdata.keys() and rowdata["_step"] >= 99:
                if minstamp is None or configdict["logname"][21:43] >= minstamp:
                    save_key = configdict["model_save_dir"]
                    data.append((configdict["latent_dim"],
                                 aligned_notw_errors_dict[save_key][0],#rowdata["train_alignedRMSE"],
                                 aligned_notw_errors_dict[save_key][1],#rowdata["test_alignedRMSE"],
                                 rowdata["train_noiselessRMSE"],
                                 rowdata["test_noiselessRMSE"],
                                 np.log2(rowdata["train_KLD"])))
    data = np.array(data)       
    return data

In [ ]:
datasets = [create_dataset(rd) for rd in all_run_dicts]

In [ ]:
[print(d.shape[0]) for d in datasets]

In [ ]:
valid_indices = [i for i,d in enumerate(datasets) if d.shape[0]>3]

In [ ]:
valid_indices

In [ ]:
datasets = [datasets[vi] for vi in valid_indices]
all_run_dicts = [all_run_dicts[vi] for vi in valid_indices]

In [ ]:
def table_with_average_variance_line(data):
    #ax.scatter(data[:,0],data[:,1],c=color,alpha=1,marker=markerstyle)
    xvals = []
    yvals = []
    stdyvals = []
    for x,y in data:
        if x not in xvals:
            xvals.append(x)
            yvals.append(np.mean(data[data[:,0]==x,1]))
            stdyvals.append(np.std(data[data[:,0]==x,1]))
    xvals = np.array(xvals)
    yvals = np.array(yvals)
    stdyvals = np.array(stdyvals)
    order = np.argsort(xvals)

    return(xvals, yvals, stdyvals)

In [ ]:

pcaresults = np.load("../fork_pca_results.npy")
pcaresults = pcaresults[pcaresults[:,0]>0]
pcaresults = pcaresults[pcaresults[:,0]<17]
datasets.append(pcaresults)

In [ ]:
pcaresults

In [ ]:
import matplotlib.lines as mlines 
#https://stackoverflow.com/questions/47391702/how-to-make-a-colored-markers-legend-from-scratch

In [ ]:
def save_table(namebase,toplot = None):

    if toplot is None:
        toplot = range(len(datasets))
    ratedat = []
    for i in toplot:
        dataval = datasets[i]
        if dataval.shape[1] > 5:
            returnval = table_with_average_variance_line(dataval[:,(0,5)])
            ratedat.append(returnval)
            
    traindat = []
    for i in toplot:
        dataval = datasets[i]
        if dataval.shape[1] > 5:
            returnval = table_with_average_variance_line(dataval[:,(0,1)])
            traindat.append(returnval)
            
    testdat = []
    for i in toplot:
        dataval = datasets[i]
        if dataval.shape[1] > 5:
            returnval = table_with_average_variance_line(dataval[:,(0,2)])
            testdat.append(returnval)
            
    
    return (ratedat, traindat, testdat)

In [ ]:
def is_base_config(testdict,must_not_match=[]):
    checks = [("beta",0.1),
                ("scalar_timewarper_name","modeled_scalar_timewarper"),
                ("decoder_name","functional_decoder_complicated"),
                ("dec_complicated_function_hidden_dims",[200]),
                ("training_data_added_timing_noise",0.1),
                ("dec_use_tanh",1)
             ]
    for check in checks:
        if check[0] in must_not_match:
            if safe_check_required_key_value(testdict,check[0],check[1]):
                return False
        elif not safe_check_required_key_value(testdict,check[0],check[1]):
            return False
    return True

In [ ]:
np.array([is_base_config(d) for d in all_run_dicts])

In [ ]:
np.sum(np.array([is_base_config(d) for d in all_run_dicts]))

In [ ]:
[d for d in all_run_dicts 
 if is_base_config(d,["decoder_name","scalar_timewarper_name","dec_complicated_function_hidden_dims","dec_use_tanh"])]


In [ ]:
base_config_index = np.where(np.array([is_base_config(d,"dec_use_tanh") for d in all_run_dicts]))[0].item()
notimewarp_index = np.where(np.array([is_base_config(d,["scalar_timewarper_name","dec_use_tanh"]) for d in all_run_dicts]))[0].item()
conv_index = np.where(np.array([is_base_config(d,["decoder_name","scalar_timewarper_name","dec_complicated_function_hidden_dims","dec_use_tanh"]) for d in all_run_dicts]))[0].item()

beta_inds = list(np.where(np.array([is_base_config(d,["beta","dec_use_tanh"]) for d in all_run_dicts]))[0])
no_tw_beta_inds = list(np.where(np.array([is_base_config(d,["beta","scalar_timewarper_name","dec_use_tanh"]) for d in all_run_dicts]))[0])
conv_beta_inds = list(np.where(np.array([is_base_config(d,["beta","decoder_name","scalar_timewarper_name","dec_complicated_function_hidden_dims","dec_use_tanh"]) for d in all_run_dicts]))[0])

ria_ind = list(np.where(np.array([is_base_config(d,["decoder_name","scalar_timewarper_name","dec_complicated_function_hidden_dims"]) for d in all_run_dicts]))[0])[0].item()
ria_beta_inds = list(np.where(np.array([is_base_config(d,["beta","decoder_name","scalar_timewarper_name","dec_complicated_function_hidden_dims"]) for d in all_run_dicts]))[0])[0].item()

print(ria_ind)

print(len(all_run_dicts))

In [ ]:
all_run_dicts[ria_ind]

In [ ]:
toplots=[base_config_index] + beta_inds+ no_tw_beta_inds + [notimewarp_index] + conv_beta_inds + [conv_index] + [ria_ind,ria_beta_inds]

In [ ]:
(rates, trains, tests) = save_table("beta",toplots)

In [ ]:
trains

In [ ]:
print([all_run_dicts[i] for i in toplots])

In [ ]:
names = ["TimewarpVAE" if all_run_dicts[i]["scalar_timewarper_name"] == "modeled_scalar_timewarper" else 
         "beta-VAE" if all_run_dicts[i]["decoder_name"] == "convolutional_decoder_upsampling" else 
         "RateInvariantAE" if all_run_dicts[i]["decoder_name"] == "rate_invariant_conv" else
         "NoTimewarp" for i in toplots]
betas = np.array([all_run_dicts[i]["beta"] for i in toplots])

In [ ]:
betas

In [ ]:
outstring = ""
prevname = None
almost_sorted = np.flip(np.lexsort((-betas,names)))
sorted_indices = [almost_sorted[j] for j in [3,4,6,7,0,1,8,9]]
for i in sorted_indices:
    if betas[i] == 0.0001:
        continue
    (n,b,r,trai,test) = (names[i], betas[i], rates[i], trains[i],tests[i])
    if prevname is not None and n == prevname:
        thisname = ""
    else:
        thisname = n
    openbrace="{"
    closebrace="}"
    if n == "TimewarpVAE" and b == 0.1:
        outstring += (f"{thisname} & {b} & {r[1].item():.3f} & "
                    +f"\\textbf{openbrace}{trai[1].item():.3f} $\\pm$ {3*trai[2].item():.3f}{closebrace} & "
                    +f"{test[1].item():.3f} $\\pm$ {3*test[2].item():.3f} \\\\\n")
    elif n == "TimewarpVAE" and b == 0.01:
        outstring += (f"{thisname} & {b} & {r[1].item():.3f} & "
                    +f"{trai[1].item():.3f} $\\pm$ {3*trai[2].item():.3f} & "
                    +f"\\textbf{openbrace}{test[1].item():.3f} $\\pm$ {3*test[2].item():.3f}{closebrace} \\\\\n")
    else:
        outstring += (f"{thisname} & {b} & {r[1].item():.3f} & "
                    +f"{trai[1].item():.3f} $\\pm$ {3*trai[2].item():.3f} & "
                    +f"{test[1].item():.3f} $\\pm$ {3*test[2].item():.3f}\\\\\n")
    prevname = n
print(outstring)

In [ ]:
with open("forkResultsTable.tex", "w") as text_file:
    text_file.write(outstring[:-2]) # the last line needs its return to be hardcoded into the main tex file